### Pneumonia Detection on Chest X-ray Images Using Deep Learning

The dataset of this project is obtained from the https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia

### Data set:
The dataset is organized into 3 folders (train, test, val) and contains subfolders of each image category (Pneumonia / Normal). There are 5,863 X-Ray images (JPEG) and 2 categories(Pneumonia/Normal)

Chest X-ray images (anterior-posterior) were selected from retrospective cohorts of pediatric patients of one to five years old from Guangzhou Women and Children’s Medical Center, Guangzhou. All chest X-ray imaging was performed as part of patients’ routine clinical care.

For the analysis of chest x-ray images, all chest radiographs were initially screened for quality control by removing all low quality or unreadable scans. The diagnoses for the images were then graded by two expert physicians before being cleared for training the AI system. In order to account for any grading errors, the evaluation set was also checked by a third expert.

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt


import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense,Dropout,Flatten, Conv2D,MaxPooling2D,BatchNormalization,Input
from tensorflow.keras.regularizers import L1

## Architecture of Model:
### input:
- input: (224,224,1)

### convolution Layers:
- conv1: 32 size (7,7) # kernel size should be always odd numbers only 
- Batch normalization
- pool1: (3,3)
- dropout layer-1: 20% # to address overfitiing 

- conv2: 64 size (7,7) # kernel size should be always odd numbers only 
- Batch normalization
- pool2: (3,3)
- dropout layer-2: 20% # to address overfitiing 

- conv3: 128 size (7,7) # kernel size should be always odd numbers only 
- Batch normalization
- pool3: (3,3)
- dropout layer-3: 20%  # to address overfitiing 


### Flatten layer:
- Flatten: 

### Fully connecte Layer:
- Dense1: with 1024 nodes
- Batch normalization
- dropout layer: 25% # to address overfitiing 

- Dense2: with 512 nodes
- Batch normalization 
- dropout layer: 25% # to address overfitiing 

- Dense3: with 256 nodes
- Batch normalization 
- dropout layer: 25% # to address overfitiing

- Dense4: with 64 nodes
- Batch normalization 
- dropout layer: 25% # to address overfitiing

- Dense(**output**):with 2 nodes


In [3]:
# input laye
x=Input(shape=(224,224,1)) 

# conv1
conv1=Conv2D(filters=32,kernel_size=(7,7),activation='relu',padding='same',name='conv1')(x)
bn_conv1=BatchNormalization(name='bn_conv1')(conv1)
pool1=MaxPooling2D(pool_size=(3,3),name='pool1')(bn_conv1)
dr_conv1=Dropout(rate=0.2)(pool1)

# conv2
conv2=Conv2D(filters=64,kernel_size=(7,7),activation='relu',padding='same',name='conv2')(dr_conv1)
bn_conv2=BatchNormalization(name='bn_conv2')(conv2)
pool2=MaxPooling2D(pool_size=(3,3),name='pool2')(bn_conv2)
dr_conv2=Dropout(rate=0.2)(pool2)

# conv3
conv3=Conv2D(filters=128,kernel_size=(7,7),activation='relu',padding='same',name='conv3')(dr_conv2)
bn_conv3=BatchNormalization(name='bn_conv3')(conv3)
pool3=MaxPooling2D(pool_size=(3,3),name='pool3')(bn_conv3)
dr_conv3=Dropout(rate=0.2)(pool3)

# flatten layer
flatten=Flatten()(dr_conv3)

# dense1
dense1=Dense(1024,activation='relu',kernel_regularizer=L1(l1=0.01),name='dense1')(flatten)
bn1=BatchNormalization(name='bn1_d1')(dense1)
dr1=Dropout(0.25,name='dr1_d1')(bn1) 

# dense2
dense2=Dense(512,activation='relu',kernel_regularizer=L1(l1=0.01),name='dense2')(dr1)
bn2=BatchNormalization(name='bn2_d2')(dense2)
dr2=Dropout(0.25,name='dr2_d2')(bn2)

# dense3
dense3=Dense(256,activation='relu',kernel_regularizer=L1(l1=0.01),name='dense3')(dr2)
bn3=BatchNormalization(name='bn3_d3')(dense3)
dr3=Dropout(0.25,name='dr3_d3')(bn3)

# dense4
dense4=Dense(64,activation='relu',kernel_regularizer=L1(l1=0.01),name='dense4')(dr3)
bn4=BatchNormalization(name='bn4_d4')(dense4)
dr4=Dropout(0.25,name='dr4_d4')(bn4)

# output
output=Dense(2,activation='softmax',name='output')(dr4)

# model
model_xray=Model(inputs=x,outputs=output) 


In [4]:
model_xray.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 1)]     0         
                                                                 
 conv1 (Conv2D)              (None, 224, 224, 32)      1600      
                                                                 
 bn_conv1 (BatchNormalizati  (None, 224, 224, 32)      128       
 on)                                                             
                                                                 
 pool1 (MaxPooling2D)        (None, 74, 74, 32)        0         
                                                                 
 dropout_3 (Dropout)         (None, 74, 74, 32)        0         
                                                                 
 conv2 (Conv2D)              (None, 74, 74, 64)        100416    
                                                           

In [5]:
model_xray.compile(optimizer='rmsprop',
                   loss='categorical_crossentropy',
                  metrics=['accuracy'])

### Creating data generators:

In [6]:
# libraies for image Augmentation
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
# data preparation
# Generate batches of tensor image data with real-time data augmentation.
train_data_gen=ImageDataGenerator(
    rotation_range=20,
    #rescale=1./255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_data_gen=ImageDataGenerator(
    rotation_range=20,
    #rescale=1./255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest') 

### Loading the data into DataGenerator 
The method ***"flow_from_directory"*** loads the data recursively by going to the directory by directory if our main directory is in a hierarchical fashion.

In [7]:
width=224
height=224
batch_size=32
train_dir=r'F:/YNaiduBabu/DL_by_LalithSachan/Download_Data/chest_xray/train'
test_dir=r'F:/YNaiduBabu/DL_by_LalithSachan/Download_Data/chest_xray/test'

In [11]:
# generating data and passing as batches with specific trget size
train_generator=train_data_gen.flow_from_directory(
    train_dir,
    target_size=(height,width),
    color_mode='grayscale',# it genrates image of (height, width,3) because colour image: 'rgb'
    batch_size=batch_size,
    class_mode='categorical')

validation_generator=validation_data_gen.flow_from_directory(
    test_dir,
    target_size=(height,width),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical') 

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [13]:
from sklearn.utils import class_weight

cw = class_weight.compute_class_weight(class_weight='balanced', 
                                       classes=np.unique(train_generator.classes),
                                       y=train_generator.classes)

cw_dict = dict(enumerate(cw))
cw_dict

{0: 1.9448173005219984, 1: 0.6730322580645162}

### Training:

In [23]:
import os
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [24]:
EPOCHS =50
STEPS_PER_EPOCH= train_generator.n//train_generator.batch_size
VALIDATION_STEPS=validation_generator.n//train_generator.batch_size

In [25]:
early_stoping=EarlyStopping(monitor='val_loss',
              patience=10)

In [26]:
# creating new folder if it doesnot exists
outputFolder='./chest_xray_model_from_scratch_output'
if not os.path.exists(outputFolder):
  os.makedirs(outputFolder)

In [29]:
# ModelCheckpoint: Callback to save the Keras model or model weights at some frequency.
file_path=outputFolder+'/weights-{epoch:02d}-{loss:.4f}-{accuracy:.4f}-{val_accuracy:.4f}.h5'
checkpoint=ModelCheckpoint(filepath=file_path,
                           save_weights_only=True,
                           monitor='val_accurary',
                           mode='max',
                           save_best_only=False,
                           #save_freq=41
                          )

In [30]:
model_xray.fit(
    train_generator,
    #class_weight=cw_dict,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=validation_generator,
    validation_steps=VALIDATION_STEPS, 
    callbacks=[early_stoping,checkpoint],
    verbose=1) 

Epoch 1/50
163/163 [==============================] - 136s 835ms/step - loss: 53.2489 - accuracy: 0.7170 - val_loss: 48.4602 - val_accuracy: 0.6250
Epoch 2/50
163/163 [==============================] - 136s 831ms/step - loss: 50.4602 - accuracy: 0.7615 - val_loss: 52.0528 - val_accuracy: 0.6250
Epoch 3/50
163/163 [==============================] - 136s 830ms/step - loss: 49.4066 - accuracy: 0.7926 - val_loss: 47.9379 - val_accuracy: 0.5921
Epoch 4/50
163/163 [==============================] - 136s 835ms/step - loss: 49.2137 - accuracy: 0.8131 - val_loss: 49.5655 - val_accuracy: 0.6201
Epoch 5/50
163/163 [==============================] - 150s 919ms/step - loss: 48.7328 - accuracy: 0.8252 - val_loss: 48.7016 - val_accuracy: 0.7253
Epoch 6/50
163/163 [==============================] - 168s 1s/step - loss: 48.4420 - accuracy: 0.8315 - val_loss: 48.0993 - val_accuracy: 0.6349
Epoch 7/50
163/163 [==============================] - 140s 858ms/step - loss: 48.6085 - accuracy: 0.8459 - val_loss

In [ ]:
import pickle
model_json = model_fn.to_json()

# saving the model architecture
with open("F:/YNaiduBabu/DL_by_LalithSachan/Download_Data/chest_xray/model_xray_json_v0.json", "w") as json_file:
    json_file.write(model_json)

till now we got best results **At Epoch 31** : with **training accuray: 89.95% and validation accuracy: 85.36%.** 

we saved weights of the model and model architecture for future predictions.